In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

from glob import glob

import requests # 카카오 맵 API 불러오기 위해

import csv # 마포구 데이터 csv 모듈로 리스트로 먼저 받아온 후 오류 수정, 데이터프레임화 하기 위해

## 강남구 쓰레기통

In [2]:
gangnam_cbb = pd.read_csv('data/쓰레기통/서울특별시 강남구_담배꽁초 전용 쓰레기통_20220808.csv', encoding='cp949')
gangnam_cbb.head()

,동명,설치주소,위 치
0,신사동,압구정로14길 6(신사동 548-4),코다차야 남동쪽(가게 앞)
1,신사동,강남대로152길 14(신사동 513-2),FLASH BAR 동쪽(전신주)
2,신사동,강남대로152길 29(신사동 515-12),인사동촌 남쪽(가게 앞)
3,신사동,강남대로152길 34(신사동 514-8),고향역(전신주)
4,논현1동,학동로2길 11 (취영루) 맞은편,영풍빌딩 주차장 옆


In [3]:
# 23, 24번째 값이 중복
gangnam_cbb[22:30]

,동명,설치주소,위 치
22,삼성1동,테헤란로81길 46,가게앞 모퉁이
23,삼성1동,테헤란로83길 12,제일주차장앞 보도상
24,삼성1동,테헤란로83길 12,제일주차장앞 보도상
25,삼성1동,삼성로92길 28,도로 모퉁이
26,삼성1동,삼성로96길 15,오징어세상가게 옆골목도로변
27,삼성1동,삼성로104길 22,주소지앞 모퉁이 보도상
28,삼성1동,봉은사로82길 32,주소지 앞 보도상
29,삼성1동,삼성로104길 17,주소지 앞 보도상


In [4]:
gangnam_cbb.shape

(45, 3)

In [5]:
gangnam_cbb = gangnam_cbb.drop_duplicates(ignore_index=True)

In [6]:
gangnam_cbb.shape

(44, 3)

In [7]:
# 설치주소 + (부연 설명) 있는 것을 제거하기 위함
gangnam_cbb['설치주소'] = gangnam_cbb['설치주소'].apply(lambda x: x.split('(')[0])

In [9]:
# 설치 주소 내 맞은편, 앞 등과 같은 단어 제거
a = gangnam_cbb['설치주소'].str.split(' ')
a.head()

0      [압구정로14길, 6]
1    [강남대로152길, 14]
2    [강남대로152길, 29]
3    [강남대로152길, 34]
4     [학동로2길, 11, ]
Name: 설치주소, dtype: object

In [10]:
for i in range(len(a)):
    if a[i][-1] == '맞은편':
        a[i] = a[i][:-1]
    elif a[i][-1] == '앞':
        a[i] = a[i][:-1]

In [11]:
for i in range(len(a)):
    if len(a[i]) == 1:
        a[i] = '서울특별시 강남구 ' + a[i][0]
    else:
        a[i] = '서울특별시 강남구 ' + a[i][0] + ' ' + a[i][1]

In [12]:
a

0       서울특별시 강남구 압구정로14길 6
1     서울특별시 강남구 강남대로152길 14
2     서울특별시 강남구 강남대로152길 29
3     서울특별시 강남구 강남대로152길 34
4        서울특별시 강남구 학동로2길 11
5        서울특별시 강남구 학동로2길 19
6        서울특별시 강남구 학동로2길 47
7       서울특별시 강남구 봉은사로1길 35
8       서울특별시 강남구 봉은사로1길 32
9       서울특별시 강남구 봉은사로1길 16
10      서울특별시 강남구 봉은사로1길 16
11       서울특별시 강남구 학동로2길 47
12       서울특별시 강남구 학동로2길 13
13       서울특별시 강남구 논현동 98-6
14      서울특별시 강남구 논현동 97-20
15      서울특별시 강남구 논현동 97-18
16      서울특별시 강남구 논현동 97-17
17      서울특별시 강남구 논현동 98-12
18      서울특별시 강남구 논현동 98-12
19     서울특별시 강남구 논현동 105-12
20      서울특별시 강남구 압구정로36길 8
21     서울특별시 강남구 삼성로100길 24
22     서울특별시 강남구 테헤란로81길 46
23     서울특별시 강남구 테헤란로83길 12
24      서울특별시 강남구 삼성로92길 28
25      서울특별시 강남구 삼성로96길 15
26     서울특별시 강남구 삼성로104길 22
27     서울특별시 강남구 봉은사로82길 32
28     서울특별시 강남구 삼성로104길 17
29     서울특별시 강남구 삼성로104길 28
30       서울특별시 강남구  테헤란로64길
31       서울특별시 강남구  테헤란로64길
32        서울특별시 강남구  선릉로86길
33        서울특별시 강남구  삼성로85길
34        서울특별시 강남구  선릉로90길
35        서울특별시 강남구 

In [13]:
gangnam_cbb['설치주소'] = a

In [14]:
gangnam_cbb.head()

,동명,설치주소,위 치
0,신사동,서울특별시 강남구 압구정로14길 6,코다차야 남동쪽(가게 앞)
1,신사동,서울특별시 강남구 강남대로152길 14,FLASH BAR 동쪽(전신주)
2,신사동,서울특별시 강남구 강남대로152길 29,인사동촌 남쪽(가게 앞)
3,신사동,서울특별시 강남구 강남대로152길 34,고향역(전신주)
4,논현1동,서울특별시 강남구 학동로2길 11,영풍빌딩 주차장 옆


In [15]:
address = '서울특별시 강남구 압구정로14길 6'
url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(address)
headers = {"Authorization": 'KakaoAK 9b47700a217b794a8878d97b26b723be'}
results = requests.get(url, headers=headers).json()['documents']
results

[{'address': {'address_name': '서울 강남구 신사동 548-4',
   'b_code': '1168010700',
   'h_code': '1168051000',
   'main_address_no': '548',
   'mountain_yn': 'N',
   'region_1depth_name': '서울',
   'region_2depth_name': '강남구',
   'region_3depth_h_name': '신사동',
   'region_3depth_name': '신사동',
   'sub_address_no': '4',
   'x': '127.022980226111',
   'y': '37.523858489259'},
  'address_name': '서울 강남구 압구정로14길 6',
  'address_type': 'ROAD_ADDR',
  'road_address': {'address_name': '서울 강남구 압구정로14길 6',
   'building_name': '',
   'main_building_no': '6',
   'region_1depth_name': '서울',
   'region_2depth_name': '강남구',
   'region_3depth_name': '신사동',
   'road_name': '압구정로14길',
   'sub_building_no': '',
   'underground_yn': 'N',
   'x': '127.022980226111',
   'y': '37.523858489259',
   'zone_no': '06029'},
  'x': '127.022980226111',
  'y': '37.523858489259'}]

In [16]:
address = '서울특별시 강남구 삼성로92길 28'
url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(address)
headers = {"Authorization": 'KakaoAK 9b47700a217b794a8878d97b26b723be'}
results = requests.get(url, headers=headers).json()['documents']
results

[]

In [17]:
# 위경도 매칭 함수
def GetLatLng(address):
    
    url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(address) #요청할 url 주소
    headers = {"Authorization": 'KakaoAK 9b47700a217b794a8878d97b26b723be'} #REST API 키(유효한 키)
    results = requests.get(url,
                          headers=headers).json()['documents'] #카카오 API 요청
    
    result = results[0]
    name = result['address_name']
    x = result['x'] # Lng, 경도
    y = result['y'] # Lat, 위도
    data = [name, x, y, address]
    return data

In [18]:
GetLatLng('서울특별시 강남구 압구정로14길 6')

['서울 강남구 압구정로14길 6',
 '127.022980226111',
 '37.523858489259',
 '서울특별시 강남구 압구정로14길 6']

In [19]:
# '서울특별시 강남구 삼성로92길 28' 이 위경도 매칭 오류가 나서 27과 29의 위경도가 얼마나 다른가 확인
GetLatLng('서울특별시 강남구 삼성로92길 27')

['서울 강남구 삼성로92길 27',
 '127.058532226672',
 '37.5083786932357',
 '서울특별시 강남구 삼성로92길 27']

In [20]:
GetLatLng('서울특별시 강남구 삼성로92길 29')

['서울 강남구 삼성로92길 29',
 '127.058778402547',
 '37.5083178432764',
 '서울특별시 강남구 삼성로92길 29']

In [22]:
gangnamlist = []
for i in list(gangnam_cbb['설치주소'][:24]):
    gangnamlist.append(GetLatLng(i))
    print(GetLatLng(i))

['서울 강남구 압구정로14길 6', '127.022980226111', '37.523858489259', '서울특별시 강남구 압구정로14길 6']
['서울 강남구 강남대로152길 14', '127.020699344217', '37.5171506735807', '서울특별시 강남구 강남대로152길 14']
['서울 강남구 강남대로152길 29', '127.021697932358', '37.5179477021889', '서울특별시 강남구 강남대로152길 29']
['서울 강남구 강남대로152길 34', '127.02210634185', '37.5177615692586', '서울특별시 강남구 강남대로152길 34']
['서울 강남구 학동로2길 11', '127.02273341768', '37.5105392729193', '서울특별시 강남구 학동로2길 11']
['서울 강남구 학동로2길 19', '127.022960047655', '37.5100607055782', '서울특별시 강남구 학동로2길 19']
['서울 강남구 학동로2길 47', '127.023672833289', '37.5086256248304', '서울특별시 강남구 학동로2길 47']
['서울 강남구 봉은사로1길 35', '127.024049967581', '37.5070191469956', '서울특별시 강남구 봉은사로1길 35']
['서울 강남구 봉은사로1길 32', '127.024515199807', '37.5068294798978', '서울특별시 강남구 봉은사로1길 32']
['서울 강남구 봉은사로1길 16', '127.024941721878', '37.5058765776821', '서울특별시 강남구 봉은사로1길 16']
['서울 강남구 봉은사로1길 16', '127.024941721878', '37.5058765776821', '서울특별시 강남구 봉은사로1길 16']
['서울 강남구 학동로2길 47', '127.023672833289', '37.5086256248304', '서울특별시 강남구 학동

In [23]:
for i in list(gangnam_cbb['설치주소'][25:]):
    gangnamlist.append(GetLatLng(i))
    print(GetLatLng(i))

['서울 강남구 삼성로96길 15', '127.057075333082', '37.5090904790547', '서울특별시 강남구 삼성로96길 15']
['서울 강남구 삼성로104길 22', '127.056586087827', '37.5109410172372', '서울특별시 강남구 삼성로104길 22']
['서울 강남구 봉은사로82길 32', '127.055317832558', '37.5110991135466', '서울특별시 강남구 봉은사로82길 32']
['서울 강남구 삼성로104길 17', '127.055954807579', '37.5111971130935', '서울특별시 강남구 삼성로104길 17']
['서울 강남구 삼성로104길 28', '127.05702008388', '37.5112487718071', '서울특별시 강남구 삼성로104길 28']
['서울 강남구 테헤란로64길', '127.051309674582', '37.5039816325824', '서울특별시 강남구  테헤란로64길']
['서울 강남구 테헤란로64길', '127.051309674582', '37.5039816325824', '서울특별시 강남구  테헤란로64길']
['서울 강남구 선릉로86길', '127.05171067105', '37.5035948362881', '서울특별시 강남구  선릉로86길']
['서울 강남구 삼성로85길', '127.055535567476', '37.5048466687952', '서울특별시 강남구  삼성로85길']
['서울 강남구 선릉로90길', '127.05231763904', '37.5048727352622', '서울특별시 강남구  선릉로90길']
['서울 강남구 삼성로85길', '127.055535567476', '37.5048466687952', '서울특별시 강남구  삼성로85길']
['서울 강남구 선릉로90길 56', '127.053831329394', '37.5050311726932', '서울특별시 강남구 선릉로90길 56']
['서울 강남구 강남대로

In [27]:
# '서울특별시 강남구 삼성로92길 28'은 제외하고 생성
gangnamlist

[['서울 강남구 압구정로14길 6',
  '127.022980226111',
  '37.523858489259',
  '서울특별시 강남구 압구정로14길 6'],
 ['서울 강남구 강남대로152길 14',
  '127.020699344217',
  '37.5171506735807',
  '서울특별시 강남구 강남대로152길 14'],
 ['서울 강남구 강남대로152길 29',
  '127.021697932358',
  '37.5179477021889',
  '서울특별시 강남구 강남대로152길 29'],
 ['서울 강남구 강남대로152길 34',
  '127.02210634185',
  '37.5177615692586',
  '서울특별시 강남구 강남대로152길 34'],
 ['서울 강남구 학동로2길 11',
  '127.02273341768',
  '37.5105392729193',
  '서울특별시 강남구 학동로2길 11'],
 ['서울 강남구 학동로2길 19',
  '127.022960047655',
  '37.5100607055782',
  '서울특별시 강남구 학동로2길 19'],
 ['서울 강남구 학동로2길 47',
  '127.023672833289',
  '37.5086256248304',
  '서울특별시 강남구 학동로2길 47'],
 ['서울 강남구 봉은사로1길 35',
  '127.024049967581',
  '37.5070191469956',
  '서울특별시 강남구 봉은사로1길 35'],
 ['서울 강남구 봉은사로1길 32',
  '127.024515199807',
  '37.5068294798978',
  '서울특별시 강남구 봉은사로1길 32'],
 ['서울 강남구 봉은사로1길 16',
  '127.024941721878',
  '37.5058765776821',
  '서울특별시 강남구 봉은사로1길 16'],
 ['서울 강남구 봉은사로1길 16',
  '127.024941721878',
  '37.5058765776821',
  '서울특별시 강남

In [28]:
gangnam_latlng = pd.DataFrame(gangnamlist)
gangnam_latlng.columns = ['address_name', '경도', '위도', '설치주소']
gangnam_latlng.head()

,address_name,경도,위도,설치주소
0,서울 강남구 압구정로14길 6,127.022980226111,37.523858489259,서울특별시 강남구 압구정로14길 6
1,서울 강남구 강남대로152길 14,127.020699344217,37.5171506735807,서울특별시 강남구 강남대로152길 14
2,서울 강남구 강남대로152길 29,127.021697932358,37.5179477021889,서울특별시 강남구 강남대로152길 29
3,서울 강남구 강남대로152길 34,127.02210634185,37.5177615692586,서울특별시 강남구 강남대로152길 34
4,서울 강남구 학동로2길 11,127.02273341768,37.5105392729193,서울특별시 강남구 학동로2길 11


In [29]:
gangnam_m = pd.merge(gangnam_latlng, gangnam_cbb, on=['설치주소'], how='left')
gangnam_m.drop_duplicates
print(gangnam_m.shape)
gangnam_m.head()

(53, 6)


,address_name,경도,위도,설치주소,동명,위 치
0,서울 강남구 압구정로14길 6,127.022980226111,37.523858489259,서울특별시 강남구 압구정로14길 6,신사동,코다차야 남동쪽(가게 앞)
1,서울 강남구 강남대로152길 14,127.020699344217,37.5171506735807,서울특별시 강남구 강남대로152길 14,신사동,FLASH BAR 동쪽(전신주)
2,서울 강남구 강남대로152길 29,127.021697932358,37.5179477021889,서울특별시 강남구 강남대로152길 29,신사동,인사동촌 남쪽(가게 앞)
3,서울 강남구 강남대로152길 34,127.02210634185,37.5177615692586,서울특별시 강남구 강남대로152길 34,신사동,고향역(전신주)
4,서울 강남구 학동로2길 11,127.02273341768,37.5105392729193,서울특별시 강남구 학동로2길 11,논현1동,영풍빌딩 주차장 옆


In [30]:
gangnam_m['시군구명']='강남구'

In [31]:
gangnam_m = gangnam_m[['시군구명', '동명', 'address_name', '경도', '위도', '설치주소', '위  치']]
gangnam_m.head()

,시군구명,동명,address_name,경도,위도,설치주소,위 치
0,강남구,신사동,서울 강남구 압구정로14길 6,127.022980226111,37.523858489259,서울특별시 강남구 압구정로14길 6,코다차야 남동쪽(가게 앞)
1,강남구,신사동,서울 강남구 강남대로152길 14,127.020699344217,37.5171506735807,서울특별시 강남구 강남대로152길 14,FLASH BAR 동쪽(전신주)
2,강남구,신사동,서울 강남구 강남대로152길 29,127.021697932358,37.5179477021889,서울특별시 강남구 강남대로152길 29,인사동촌 남쪽(가게 앞)
3,강남구,신사동,서울 강남구 강남대로152길 34,127.02210634185,37.5177615692586,서울특별시 강남구 강남대로152길 34,고향역(전신주)
4,강남구,논현1동,서울 강남구 학동로2길 11,127.02273341768,37.5105392729193,서울특별시 강남구 학동로2길 11,영풍빌딩 주차장 옆


In [32]:
gangnam_m.to_csv('강남구_담배꽁초쓰레기통_위경도.csv', index=False)

## 동작구 쓰레기통

In [ ]:
dongjak_cbb = pd.read_csv('data/쓰레기통/서울특별시 동작구_담배꽁초 전용 쓰레기통 현황_20230427.csv', encoding='cp949')
dongjak_cbb.head()

In [ ]:
dongjaklist = []

In [ ]:
for i in list(dongjak_cbb['도로명주소']):
    dongjaklist.append(GetLatLng(i))
    print(GetLatLng(i))

In [ ]:
dongjak_latlng = pd.DataFrame(dongjaklist)
dongjak_latlng.columns = ['address_name', '경도', '위도', '도로명주소']
dongjak_latlng.head()

In [ ]:
dongjak_m = pd.merge(dongjak_latlng, dongjak_cbb, on='도로명주소', how='left')
dongjak_m.drop_duplicates()
print(dongjak_m.shape)
dongjak_m.head()

In [ ]:
dongjak_m['시군구명']='동작구'

In [ ]:
dongjak_m.columns

In [ ]:
dongjak_m = dongjak_m.drop(columns=['연번', '데이터 기준일자'])

In [ ]:
dongjak_m = dongjak_m.rename(columns = {'도로명주소' : '설치주소'})

In [ ]:
dongjak_m = dongjak_m[['시군구명', 'address_name', '경도', '위도', '설치주소', '설치장소(상호명)', '비고(관리연합회)']]
dongjak_m.head()

## 마포구 쓰레기통

In [ ]:
file_path = 'data\쓰레기통\서울특별시 마포구 용강동 상점가 담배꽁초수거함 설치 현황_20220805.csv'
f = open(file_path, 'rt')
reader = csv.reader(f)

csv_list = []
for l in reader:
    csv_list.append(l)
f.close()

df = pd.DataFrame(csv_list)
df

In [ ]:
mapo_cbb = pd.read_csv('data/쓰레기통/서울특별시 마포구 용강동 상점가 담배꽁초수거함 설치 현황_20220805.csv')
mapo_cbb.head()

In [ ]:
mapo_cbb = mapo_cbb.drop(columns=['Unnamed: 7'])

In [ ]:
mapo_cbb['도로명주소'] = mapo_cbb['도로명주소'].apply(lambda x:x.split('(')[0])
mapo_cbb.head()

In [ ]:
# 주소 변경 데이터 1
# 토정로25길 26인 것일까
# 토정로 261인 것일까...
# 육도라는 상호명이 토정로 261임
address = '서울특별시 마포구 토정로25길 261'
url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(address)
headers = {"Authorization": 'KakaoAK 9b47700a217b794a8878d97b26b723be'}
results = requests.get(url, headers=headers).json()['documents']
results

In [ ]:
# 주소 변경 데이터 2
# 석양집의 주소는 '토정로 317'
# 설치 세부 위치인 띠녹지 앞은 '토정로 35길 12' - 하지만 해당 주소로는 위경도가 안나옴
address = '서울특별시 마포구 토정로35길 12'
url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(address)
headers = {"Authorization": 'KakaoAK 9b47700a217b794a8878d97b26b723be'}
results = requests.get(url, headers=headers).json()['documents']
results

In [ ]:
mapo_cbb.loc[mapo_cbb['도로명주소']=='서울특별시 마포구 토정로25길 261'] = '서울특별시 마포구 토정로 261'

In [ ]:
# 외 1건이라면, 수량에 2개를 설정해야 하는지 고민
mapo_cbb.loc[mapo_cbb['도로명주소']=='서울특별시 마포구 토정로35길 12외 1건'] = '서울특별시 마포구 토정로 317'

In [ ]:
mapo_cbb['도로명주소'][43:]

In [ ]:
mapolist=[]
for i in list(mapo_cbb['도로명주소']):
    mapolist.append(GetLatLng(i))
    print(GetLatLng(i))

In [ ]:
mapo_latlng = pd.DataFrame(mapolist)
mapo_latlng.columns = ['address_name', '경도', '위도', '도로명주소']
mapo_latlng.head()

In [ ]:
mapo_m = pd.merge(mapo_latlng, mapo_cbb, on=['도로명주소'], how='left')
mapo_m.drop_duplicates()
print(mapo_m.shape)
mapo_m.head()

In [ ]:
mapo_m = mapo_m.drop(columns=['시도명', '데이터기준일자'])

In [ ]:
mapo_m = mapo_m[['시군구명', 'address_name', '경도', '위도', '도로명주소', '상호명', '수량', '세부위치']]
mapo_m.head()

## 서초구 쓰레기통

In [ ]:
seocho_cbb = pd.read_csv('data/쓰레기통/서울특별시 서초구_담배꽁초수거함 현황_20220816.csv', encoding='cp949')
seocho_cbb.head()

In [ ]:
seocho_cbb = seocho_cbb.rename(columns={'자치구명':'시군구명', '해당동':'동명', '설치위치':'설치주소'})

In [ ]:
seocho_cbb[['시군구명', '동명', '경도', '위도', '설치주소', '설치주체', '청소관리', '설치일자', '비고']]

## 양천구 쓰레기통

In [33]:
yangcheon_cbb = pd.read_csv('data/쓰레기통/서울특별시 양천구_담배꽁초쓰레기통_20220816.csv', encoding='cp949')
yangcheon_cbb.head()

,연번,설치장소,도로명주소,설치일자,데이터기준일자
0,1,현대41타워앞 인도,서울시 양천구 목동동로 293,2022-05-01,2022-08-16
1,2,파리지앙A앞 공용주차장,서울시 양천구 목동서로 57,2022-05-01,2022-08-16
2,3,보성상가옆 먹자골목,서울시 양천구 오목로 232,2022-05-01,2022-08-16
3,4,신정네거리역 뒤 먹자골목,서울시 양천구 중앙로 276,2022-05-01,2022-08-16
4,5,농협 옆 골목,서울시 양천구 오목로 325,2022-05-01,2022-08-16


In [34]:
yangcheon_cbb = yangcheon_cbb.drop(columns=['연번', '설치일자', '데이터기준일자'])

In [35]:
yangcheonlist = []
for i in list(yangcheon_cbb['도로명주소']):
    try:
        yangcheonlist.append(GetLatLng(i))
    except Exception as e:
        print(f'주소 검색 오류: {e}')
        yangcheonlist.append(None)

In [36]:
[i for i in yangcheonlist if i==None]

[]

In [37]:
yangcheon_latlng = pd.DataFrame(yangcheonlist)
yangcheon_latlng.columns = ['address_name', '경도', '위도', '도로명주소']
yangcheon_latlng.head()

,address_name,경도,위도,도로명주소
0,서울 양천구 목동동로 293,126.875771989415,37.5281783139269,서울시 양천구 목동동로 293
1,서울 양천구 목동서로 57,126.879863427781,37.5366321308561,서울시 양천구 목동서로 57
2,서울 양천구 오목로 232,126.863206293208,37.5257642505039,서울시 양천구 오목로 232
3,서울 양천구 중앙로 276,126.85288429171,37.5208361584987,서울시 양천구 중앙로 276
4,서울 양천구 오목로 325,126.873626981623,37.5251074073334,서울시 양천구 오목로 325


In [38]:
yangcheon_m = pd.merge(yangcheon_latlng, yangcheon_cbb, on=['도로명주소'], how='left')
yangcheon_m.drop_duplicates
print(yangcheon_m.shape)
yangcheon_m.head()

(12, 5)


,address_name,경도,위도,도로명주소,설치장소
0,서울 양천구 목동동로 293,126.875771989415,37.5281783139269,서울시 양천구 목동동로 293,현대41타워앞 인도
1,서울 양천구 목동서로 57,126.879863427781,37.5366321308561,서울시 양천구 목동서로 57,파리지앙A앞 공용주차장
2,서울 양천구 오목로 232,126.863206293208,37.5257642505039,서울시 양천구 오목로 232,보성상가옆 먹자골목
3,서울 양천구 중앙로 276,126.85288429171,37.5208361584987,서울시 양천구 중앙로 276,신정네거리역 뒤 먹자골목
4,서울 양천구 오목로 325,126.873626981623,37.5251074073334,서울시 양천구 오목로 325,농협 옆 골목


In [39]:
yangcheon_m['시군구명'] = '양천구'

In [40]:
yangcheon_m[['시군구명', 'address_name', '경도', '위도', '도로명주소', '설치장소']]

,시군구명,address_name,경도,위도,도로명주소,설치장소
0,양천구,서울 양천구 목동동로 293,126.875771989415,37.5281783139269,서울시 양천구 목동동로 293,현대41타워앞 인도
1,양천구,서울 양천구 목동서로 57,126.879863427781,37.5366321308561,서울시 양천구 목동서로 57,파리지앙A앞 공용주차장
2,양천구,서울 양천구 오목로 232,126.863206293208,37.5257642505039,서울시 양천구 오목로 232,보성상가옆 먹자골목
3,양천구,서울 양천구 중앙로 276,126.85288429171,37.5208361584987,서울시 양천구 중앙로 276,신정네거리역 뒤 먹자골목
4,양천구,서울 양천구 오목로 325,126.873626981623,37.5251074073334,서울시 양천구 오목로 325,농협 옆 골목
5,양천구,서울 양천구 오목로 329,126.874124388629,37.5249139528398,서울시 양천구 오목로 329,오목교역 2번출구 옆
6,양천구,서울 양천구 목동동로 71,126.862972876451,37.5155139074803,서울시 양천구 목동동로 71,맥도날드 공용주차장
7,양천구,서울 양천구 목동로25길 2,126.863545739883,37.5279664201806,서울시 양천구 목동로25길 2,마스터그레이드 커피 앞
8,양천구,서울 양천구 신정중앙로 103,126.863371926417,37.5265013713269,서울시 양천구 신정중앙로 103,버스 안다니는 길 먹자골목
9,양천구,서울 양천구 오목로 153,126.854492996755,37.5250240257691,서울시 양천구 오목로 153,신정역 1번출구 GS 편의점 옆 골목


In [41]:
yangcheon_m.to_csv('data/쓰레기통_위경도/양천구_담배꽁초쓰레기통_위경도.csv', index=False)

## 영등포구 쓰레기통

In [117]:
yeongdeung_cbb = pd.read_csv('data/쓰레기통/서울특별시 영등포구_담배꽁초전용쓰레기통설치현황_20220803.csv', encoding='cp949')
yeongdeung_cbb.head()

,연번,동명,주소,비고,데이터기준일
0,1,당산1동,서울특별시 영등포구 당산로32길 1-6,NaN,2022-08-03
1,2,당산1동,서울특별시 영등포구 국회대로36길 7-3,NaN,2022-08-03
2,3,당산1동,서울특별시 영등포구 당산로32길 5 이차돌,NaN,2022-08-03
3,4,당산1동,서울특별시 영등포구 국회대로34길 4 가화,NaN,2022-08-03
4,5,당산1동,서울특별시 영등포구 국회대로34길 3,NaN,2022-08-03


In [118]:
yeongdeung_cbb = yeongdeung_cbb.drop(columns=['연번', '데이터기준일'])

In [119]:
yeongdeung_cbb.shape

(241, 3)

In [120]:
# 쓰레기통 주소만 확인하기 위해 개수는 고려하지 않고 중복 제거
yeongdeung_cbb.drop_duplicates(subset='주소', keep='first', inplace=True, ignore_index=True)

In [121]:
yeongdeung_cbb.shape

(206, 3)

In [140]:
yeongdeunglist = []
for i in list(yeongdeung_cbb['주소']):
    try:
        yeongdeunglist.append(GetLatLng(i))
    except Exception as e:
        print(f'주소 검색 오류 {i}: {e}')
        yeongdeunglist.append(None)

In [139]:
# 주소 수정1
# 서울특별시 영등포구 당산로36길 14는 없고, 13과 15가 있음
# 서울특별시 영등포구 당산로38길 14가 있음. 일단 38길 14로 수정...
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 당산로36길 14', '주소'] = '서울특별시 영등포구 당산로38길 14'

# 주소 수정2
# 49길 2, 49길 4 있음. 49길 3 없음
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 당산로49길3-3', '주소'] = '서울특별시 영등포구 당산로49길 2'

# 주소 수정3
# 서울특별시 영등포구 대림로8가길 2 가 있음
# 서울특별시 영등포구 대림로8길 3이 있음
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 대림로8길 2', '주소'] = '서울특별시 영등포구 대림로8가길 2'

# 주소 수정4
# 서울특별시 영등포구 디지털로 344 또는 346 있음
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 디지털로 345', '주소'] = '서울특별시 영등포구 디지털로 344'

# 주소 수정5
# 서울특별시 영등포구 도림로40길 의 가장 큰 주소 번호는 25. 도림로 40길로 변경
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 도림로40길 29', '주소'] = '서울특별시 영등포구 도림로40길'

# 주소 수정6 서울특별시 영등포구 문래 로데오 거리 주변
# 로데오거리가 위치한 문래동3가로 변경
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 문래 로데오 거리 주변', '주소'] = '서울특별시 영등포구 문래동3가'

# 주소 수정7 서울특별시 영등포구 RUST 빵집(경인로79길 15)
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 RUST 빵집(경인로79길 15)', '주소'] = '서울특별시 영등포구 경인로79길 15'

# 주소 수정8 서울특별시 영등포구 퇴근길호프(도림로133길14-1)
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 퇴근길호프(도림로133길14-1)', '주소'] = '서울특별시 영등포구 도림로133길14-1'

# 주소 수정9 서울특별시 영등포구 골목집(도림로139가길11)
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 골목집(도림로139가길11)', '주소'] = '서울특별시 영등포구 도림로139가길11'

# 주소 수정10 서울특별시 영등포구 도림 로 98길 308 (성락교회 맞은편 과일가게와 공사현장 사이골목)
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 도림 로 98길 308 (성락교회 맞은편 과일가게와 공사현장 사이골목)', '주소'] = '서울특별시 영등포구 도림로 308'

# 주소 수정11 서울특별시 영등포구 도림로63길 20
# 아파트 단지로 변함. 제거가 나아보임
yeongdeung_cbb.drop(yeongdeung_cbb[yeongdeung_cbb['주소']=='서울특별시 영등포구 도림로63길 20'].index, inplace=True)


# 주소 수정12 서울특별시 영등포구 신풍로 13 3×5기사식당
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 신풍로 13 3×5기사식당', '주소'] = '서울특별시 영등포구 신풍로 13'

# 주소 수정13 서울특별시 영등포구 대방천로 191-1
# 근사값 191
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 대방천로 191-1', '주소'] = '서울특별시 영등포구 대방천로 191'

# 주소 수정14 서울특별시 영등포구 대방천로 169
# 근사값 168
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 대방천로 169', '주소'] = '서울특별시 영등포구 대방천로 168'

# 주소 수정15 서울특별시 영등포구 선유로25길 태양 빌딩
# 근사값 선유로 25길 7
# 태양 빌딩 ' '빈 띄어쓰기 존재
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 선유로25길 태양 빌딩 ', '주소'] = '서울특별시 영등포구 선유로25길 7'

# 주소 수정16 서울특별시 영등포구 문래로 205
# 근사값 203
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 문래로 205', '주소'] = '서울특별시 영등포구 문래로 203'

# 주소 수정17 서울특별시 영등포구 영등포로33길 16(스위트빌) 1
# 스위트빌 주소는 33길 14
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 영등포로33길 16(스위트빌) 1', '주소'] = '서울특별시 영등포구 영등포로33길 14'

# 주소 수정18 서울특별시 영등포구 영등포로33길 16(스위트빌) 2
# 중복 제거
yeongdeung_cbb.drop(yeongdeung_cbb[yeongdeung_cbb['주소']=='서울특별시 영등포구 영등포로33길 16(스위트빌) 2'].index, inplace=True)

# 주소 수정19 서울특별시 영등포구 경인로112길 4
# 근사값 114가길 4
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 경인로112길 4', '주소'] = '서울특별시 영등포구 경인로114가길 4'

In [142]:
yeongdeung_latlng = pd.DataFrame(yeongdeunglist)
yeongdeung_latlng.columns = ['address_name', '경도', '위도', '주소']
yeongdeung_latlng.head()

,address_name,경도,위도,주소
0,서울 영등포구 당산로32길 1-6,126.897266795561,37.5267406065353,서울특별시 영등포구 당산로32길 1-6
1,서울 영등포구 국회대로36길 7-3,126.898123773058,37.5265607881376,서울특별시 영등포구 국회대로36길 7-3
2,서울 영등포구 당산로32길 5,126.897473959605,37.5264757118622,서울특별시 영등포구 당산로32길 5 이차돌
3,서울 영등포구 국회대로34길 4,126.897451796745,37.5272118109642,서울특별시 영등포구 국회대로34길 4 가화
4,서울 영등포구 국회대로34길 3,126.897657998692,37.527154776227,서울특별시 영등포구 국회대로34길 3


In [143]:
yeongdeung_m = pd.merge(yeongdeung_latlng, yeongdeung_cbb, on=['주소'], how='left')
yeongdeung_m.drop_duplicates
print(yeongdeung_m.shape)
yeongdeung_m.head()

(204, 6)


,address_name,경도,위도,주소,동명,비고
0,서울 영등포구 당산로32길 1-6,126.897266795561,37.5267406065353,서울특별시 영등포구 당산로32길 1-6,당산1동,NaN
1,서울 영등포구 국회대로36길 7-3,126.898123773058,37.5265607881376,서울특별시 영등포구 국회대로36길 7-3,당산1동,NaN
2,서울 영등포구 당산로32길 5,126.897473959605,37.5264757118622,서울특별시 영등포구 당산로32길 5 이차돌,당산1동,NaN
3,서울 영등포구 국회대로34길 4,126.897451796745,37.5272118109642,서울특별시 영등포구 국회대로34길 4 가화,당산1동,NaN
4,서울 영등포구 국회대로34길 3,126.897657998692,37.527154776227,서울특별시 영등포구 국회대로34길 3,당산1동,NaN


In [144]:
yeongdeung_m['시군구명']='영등포구'

In [147]:
yeongdeung_m[['시군구명', '동명', 'address_name', '경도', '위도', '주소', '비고']]

,시군구명,동명,address_name,경도,위도,주소,비고
0,영등포구,당산1동,서울 영등포구 당산로32길 1-6,126.897266795561,37.5267406065353,서울특별시 영등포구 당산로32길 1-6,NaN
1,영등포구,당산1동,서울 영등포구 국회대로36길 7-3,126.898123773058,37.5265607881376,서울특별시 영등포구 국회대로36길 7-3,NaN
2,영등포구,당산1동,서울 영등포구 당산로32길 5,126.897473959605,37.5264757118622,서울특별시 영등포구 당산로32길 5 이차돌,NaN
3,영등포구,당산1동,서울 영등포구 국회대로34길 4,126.897451796745,37.5272118109642,서울특별시 영등포구 국회대로34길 4 가화,NaN
4,영등포구,당산1동,서울 영등포구 국회대로34길 3,126.897657998692,37.527154776227,서울특별시 영등포구 국회대로34길 3,NaN
...,...,...,...,...,...,...,...
199,영등포구,영등포동,서울 영등포구 경인로114가길 4,126.914000440734,37.5181395828305,서울특별시 영등포구 경인로114가길 4,NaN
200,영등포구,영등포본동,서울 영등포구 영신로15길 7,126.907613549018,37.5137102024436,서울특별시 영등포구 영신로15길 7 현대기술학원 정문,NaN
201,영등포구,영등포본동,서울 영등포구 영신로15길 7,126.907613549018,37.5137102024436,서울특별시 영등포구 영신로15길 7 현대기술학원 후문,NaN
202,영등포구,영등포본동,서울 영등포구 영신로17길,126.907247554301,37.5138686724654,서울특별시 영등포구 영신로17길,NaN


In [148]:
yeongdeung_m.to_csv('data/쓰레기통_위경도/영등포구_담배꽁초쓰레기통_위경도.csv', index=False)

## 중랑구 쓰레기통

In [42]:
jungnang_cbb = pd.read_csv('data/쓰레기통/서울특별시 중랑구_담배꽁초쓰레기통_20220804.csv', encoding='cp949')
jungnang_cbb

,연번,도로명,설치장소,설치 장소 유형,수거 쓰레기 종류,형태
0,1,망우로,상봉동 130-3,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
1,2,동일로,묵동 122-361 앞,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
2,3,공릉로,묵1동 주민센터 앞,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
3,4,중랑역로,중랑역로 100 (디엠씨빌딩 앞),담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
4,5,동일로,동일로 101길 20-15 주민센터,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
5,6,겸재로,겸재로3길 50 서부경로당,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
6,7,겸재로,겸재로122 박가네추어탕 앞,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함


In [43]:
jungnang_cbb['설치장소'] = jungnang_cbb['설치장소'].apply(lambda x: x.split('(')[0])
jungnang_cbb.head()

,연번,도로명,설치장소,설치 장소 유형,수거 쓰레기 종류,형태
0,1,망우로,상봉동 130-3,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
1,2,동일로,묵동 122-361 앞,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
2,3,공릉로,묵1동 주민센터 앞,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
3,4,중랑역로,중랑역로 100,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
4,5,동일로,동일로 101길 20-15 주민센터,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함


In [44]:
a = jungnang_cbb['설치장소'].str.split(' ')
a

0                [상봉동, 130-3]
1            [묵동, 122-361, 앞]
2              [묵1동, 주민센터, 앞]
3               [중랑역로, 100, ]
4    [동일로, 101길, 20-15, 주민센터]
5          [겸재로3길, 50, 서부경로당]
6         [겸재로122, 박가네추어탕, 앞]
Name: 설치장소, dtype: object

In [45]:
for i in range(len(a)):
    if a[i][-1] == '맞은편':
        a[i] = a[i][:-1]
    elif a[i][-1] == '앞':
        a[i] = a[i][:-1]

In [46]:
for i in range(len(a)):
    if len(a[i]) == 1:
        a[i] = '서울특별시 중랑구 ' + a[i][0]
    else:
        a[i] = '서울특별시 중랑구 ' + a[i][0] + ' ' + a[i][1]

In [47]:
a

0        서울특별시 중랑구 상봉동 130-3
1       서울특별시 중랑구 묵동 122-361
2         서울특별시 중랑구 묵1동 주민센터
3         서울특별시 중랑구 중랑역로 100
4         서울특별시 중랑구 동일로 101길
5         서울특별시 중랑구 겸재로3길 50
6    서울특별시 중랑구 겸재로122 박가네추어탕
Name: 설치장소, dtype: object

In [48]:
jungnang_cbb['설치장소'] = a

In [53]:
# 오류로 주소 변경
jungnang_cbb.loc[jungnang_cbb['설치장소']=='서울특별시 중랑구 묵1동 주민센터', '설치장소'] = '서울특별시 중랑구 공릉로2가길 5 묵1동주민센터'
jungnang_cbb

,연번,도로명,설치장소,설치 장소 유형,수거 쓰레기 종류,형태
0,1,망우로,서울특별시 중랑구 상봉동 130-3,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
1,2,동일로,서울특별시 중랑구 묵동 122-361,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
2,3,공릉로,서울특별시 중랑구 공릉로2가길 5 묵1동주민센터,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
3,4,중랑역로,서울특별시 중랑구 중랑역로 100,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
4,5,동일로,서울특별시 중랑구 동일로 101길,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
5,6,겸재로,서울특별시 중랑구 겸재로3길 50,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
6,7,겸재로,서울특별시 중랑구 겸재로122 박가네추어탕,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함


In [54]:
jungnanglist = []
for i in list(jungnang_cbb['설치장소']):
    try:
        jungnanglist.append(GetLatLng(i))
    except Exception as e:
        print(f'주소 검색 오류 {i}: {e}')
        jungnanglist.append(None)

In [55]:
jungnang_latlng = pd.DataFrame(jungnanglist)
jungnang_latlng.columns = ['address_name', '경도', '위도', '설치장소']
jungnang_latlng.head()

,address_name,경도,위도,설치장소
0,서울 중랑구 상봉동 130-3,127.074681320491,37.5932328015902,서울특별시 중랑구 상봉동 130-3
1,서울 중랑구 묵동 122-361,127.078312444399,37.6092671767219,서울특별시 중랑구 묵동 122-361
2,서울 중랑구 공릉로2가길 5,127.078479291591,37.612414580688,서울특별시 중랑구 공릉로2가길 5 묵1동주민센터
3,서울 중랑구 중랑역로 100,127.077797714827,37.6024297259926,서울특별시 중랑구 중랑역로 100
4,서울 중랑구 동일로101길,127.076756397416,37.5888366116663,서울특별시 중랑구 동일로 101길


In [56]:
jungnang_m = pd.merge(jungnang_latlng, jungnang_cbb, on=['설치장소'], how='left')
jungnang_m.drop_duplicates
print(jungnang_m.shape)
jungnang_m.head()

(7, 9)


,address_name,경도,위도,설치장소,연번,도로명,설치 장소 유형,수거 쓰레기 종류,형태
0,서울 중랑구 상봉동 130-3,127.074681320491,37.5932328015902,서울특별시 중랑구 상봉동 130-3,1,망우로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
1,서울 중랑구 묵동 122-361,127.078312444399,37.6092671767219,서울특별시 중랑구 묵동 122-361,2,동일로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
2,서울 중랑구 공릉로2가길 5,127.078479291591,37.612414580688,서울특별시 중랑구 공릉로2가길 5 묵1동주민센터,3,공릉로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
3,서울 중랑구 중랑역로 100,127.077797714827,37.6024297259926,서울특별시 중랑구 중랑역로 100,4,중랑역로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
4,서울 중랑구 동일로101길,127.076756397416,37.5888366116663,서울특별시 중랑구 동일로 101길,5,동일로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함


In [57]:
jungnang_m['시군구명']='중랑구'

In [58]:
jungnang_m[['시군구명', 'address_name', '경도', '위도', '설치장소', '연번', '도로명', '설치 장소 유형',
       '수거 쓰레기 종류', '형태']]

,시군구명,address_name,경도,위도,설치장소,연번,도로명,설치 장소 유형,수거 쓰레기 종류,형태
0,중랑구,서울 중랑구 상봉동 130-3,127.074681320491,37.5932328015902,서울특별시 중랑구 상봉동 130-3,1,망우로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
1,중랑구,서울 중랑구 묵동 122-361,127.078312444399,37.6092671767219,서울특별시 중랑구 묵동 122-361,2,동일로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
2,중랑구,서울 중랑구 공릉로2가길 5,127.078479291591,37.612414580688,서울특별시 중랑구 공릉로2가길 5 묵1동주민센터,3,공릉로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
3,중랑구,서울 중랑구 중랑역로 100,127.077797714827,37.6024297259926,서울특별시 중랑구 중랑역로 100,4,중랑역로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
4,중랑구,서울 중랑구 동일로101길,127.076756397416,37.5888366116663,서울특별시 중랑구 동일로 101길,5,동일로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
5,중랑구,서울 중랑구 겸재로3길 50,127.076907543484,37.5881687878315,서울특별시 중랑구 겸재로3길 50,6,겸재로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
6,중랑구,서울 중랑구 겸재로 122,127.081347905764,37.5868055827869,서울특별시 중랑구 겸재로122 박가네추어탕,7,겸재로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함


In [59]:
jungnang_m.to_csv('data/쓰레기통_위경도/중랑구_담배꽁초쓰레기통_위경도.csv', index=False)